In [1]:
## 파이토치로 로지스틱 회귀

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [2]:
W=torch.zeros((2,1),requires_grad=True)
b=torch.zeros(1, requires_grad=True)
optimizer=optim.SGD([W,b],lr=1)
epochs=1000

for epoch in range(epochs+1):
  hypothesis=torch.sigmoid(x_train.matmul(W)+b)
  cost=F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [3]:
print(torch.sigmoid(x_train.matmul(W)+b))

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [4]:
prediction=hypothesis>torch.Tensor([0.5])
prediction

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

In [5]:
## 로지스틱 회귀 : nn.Module

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

## nn.Sequential() : nn.Module 층을 차례로 쌓을 수 있다.

model=nn.Sequential(nn.Linear(2,1),nn.Sigmoid())
model(x_train)

tensor([[0.8214],
        [0.8882],
        [0.6682],
        [0.8682],
        [0.8571],
        [0.7425]], grad_fn=<SigmoidBackward0>)

In [6]:
optimizer=optim.SGD(model.parameters(),lr=1)
epochs=1000

for epoch in range(epochs):
  hypothesis=model(x_train)
  cost=F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%10==0:
    prediction=hypothesis>=torch.Tensor([0.5])
    correct_prediction=prediction.float()==y_train
    accuracy=correct_prediction.sum().item()/len(correct_prediction)
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( epoch, epochs, cost.item(), accuracy * 100,))

Epoch    0/1000 Cost: 0.934969 Accuracy 50.00%
Epoch   10/1000 Cost: 0.434614 Accuracy 66.67%
Epoch   20/1000 Cost: 0.458813 Accuracy 66.67%
Epoch   30/1000 Cost: 0.384383 Accuracy 83.33%
Epoch   40/1000 Cost: 0.325961 Accuracy 83.33%
Epoch   50/1000 Cost: 0.274742 Accuracy 83.33%
Epoch   60/1000 Cost: 0.228289 Accuracy 100.00%
Epoch   70/1000 Cost: 0.188148 Accuracy 100.00%
Epoch   80/1000 Cost: 0.160623 Accuracy 100.00%
Epoch   90/1000 Cost: 0.146059 Accuracy 100.00%
Epoch  100/1000 Cost: 0.135921 Accuracy 100.00%
Epoch  110/1000 Cost: 0.127210 Accuracy 100.00%
Epoch  120/1000 Cost: 0.119568 Accuracy 100.00%
Epoch  130/1000 Cost: 0.112809 Accuracy 100.00%
Epoch  140/1000 Cost: 0.106788 Accuracy 100.00%
Epoch  150/1000 Cost: 0.101391 Accuracy 100.00%
Epoch  160/1000 Cost: 0.096525 Accuracy 100.00%
Epoch  170/1000 Cost: 0.092116 Accuracy 100.00%
Epoch  180/1000 Cost: 0.088101 Accuracy 100.00%
Epoch  190/1000 Cost: 0.084430 Accuracy 100.00%
Epoch  200/1000 Cost: 0.081060 Accuracy 100.00

In [7]:
## 로지스틱 회귀 : Class

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(2,1)
    self.sigmoid=nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))

model=BinaryClassifier()
optimizer=optim.SGD(model.parameters(),lr=1)
epochs=1000

for epoch in range(epochs):
  hypothesis=model(x_train)
  cost=F.binary_cross_entropy(hypothesis,y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 1.123101 Accuracy 50.00%
Epoch   10/1000 Cost: 1.020545 Accuracy 50.00%
Epoch   20/1000 Cost: 0.618273 Accuracy 83.33%
Epoch   30/1000 Cost: 0.517095 Accuracy 83.33%
Epoch   40/1000 Cost: 0.430004 Accuracy 83.33%
Epoch   50/1000 Cost: 0.346926 Accuracy 83.33%
Epoch   60/1000 Cost: 0.269744 Accuracy 83.33%
Epoch   70/1000 Cost: 0.205867 Accuracy 83.33%
Epoch   80/1000 Cost: 0.166235 Accuracy 100.00%
Epoch   90/1000 Cost: 0.148563 Accuracy 100.00%
Epoch  100/1000 Cost: 0.137939 Accuracy 100.00%
Epoch  110/1000 Cost: 0.128969 Accuracy 100.00%
Epoch  120/1000 Cost: 0.121117 Accuracy 100.00%
Epoch  130/1000 Cost: 0.114183 Accuracy 100.00%
Epoch  140/1000 Cost: 0.108015 Accuracy 100.00%
Epoch  150/1000 Cost: 0.102494 Accuracy 100.00%
Epoch  160/1000 Cost: 0.097521 Accuracy 100.00%
Epoch  170/1000 Cost: 0.093020 Accuracy 100.00%
Epoch  180/1000 Cost: 0.088926 Accuracy 100.00%
Epoch  190/1000 Cost: 0.085186 Accuracy 100.00%
Epoch  200/1000 Cost: 0.081755 Accuracy 100.00%
